In [1]:
%%time
import re
import time
import collections


print("start")

def steps(M,f,t):
    e=[(f,0)]
    past={}
    while len(e):
        p,c=e[0]
        e=e[1:]
        if p==t:
            return c
        fr,ex=M[p]
        for x in ex:
            if x not in past:
                e.append((x,c+1))
        past[p]=1
        e=sorted(e,key=lambda x: x[1],reverse=False)

#load map into dictionary
def ld():
    d=open('16.txt').read().split('\n')
    
    M={}
    flows=[]
    for x in d:
        #print (x)
        n=x.split(' ')[1]
        n=re.findall('Valve ([A-Z]*)',x)[0]
        f=int(re.findall('flow rate=([0-9]*)',x)[0])
        e=re.findall('[A-Z]+',x.split('valve')[1])
        assert(len(e)>0)
        flows.append(f)
        M[n]=(f,e)
    
    #level is too big with too much state
    #it only makes sense to visit valves
    #make simpler map
    
    valves=[]
    for k in M.keys():
        f,ex=M[k]
        if f>0:
            valves.append(k)
    valves.append('AA')
    #print(valves)
    M2={}
    
    F={}
    for a in valves:
        M2[a]={}
        
        F[a]=M[a][0]
        for b in valves:
            F[b]=M[b][0]
            if a != b:
                s=steps(M,a,b)
                M2[a][b]=s
                #print("from %s to %s in %d"%(a,b,s))
    return M2,F
    


def maxrel(M,F,tt,wl=None):
    
    if wl is None:
        wl=list(M.keys())
        
    past={}
    
    #pos,time,flow,accflow,valves
    e=[('AA',0,0,0,())]
    
    lt=time.time()
    mmm=0
    #bestcflow at time
    bfat=collections.defaultdict(int)
    while len(e):
        #print("1234")
        n=e[0]
        e=e[1:]
        
        p,t,f,a,v=n
        
        if n in past or t>tt or f<bfat[t]:
            continue
        
        if f > bfat[t]:
            bfat[t]=f
        
        #go and open valve not already opened
        for ex in M[p].keys():
            if ex not in v:
                if ex in wl:
                    wt=M[p][ex]
                    p2=ex
                    t2=t+M[p][ex]+1
                    f2=f+F[ex]
                    a2=a+f*(wt+1)
                    v2=tuple(list(v)+[ex])
                    n2=(p2,t2,f2,a2,v2)
                    e.append(n2)
        
            #e.append(n2)
        #print(len(e))   
        e=sorted(e,key=lambda x: (30-x[1])*x[2]+x[3],reverse=True)
        assert(t<=tt)
        f30=(tt-t)*f+a
        past[n]=f30
        if f30>mmm:
            mmm=f30
        
        nt=time.time()
        if nt>lt+1:
            lt=nt
            print("%6d %6d %2d %3d"%(len(e), len(past),t,mmm),end='\r')
        
        
        
        #print(past[n])
    #print("\nfin")
    return max(past.values())

def p1():
    M,F=ld()
    return maxrel(M,F,30)

print("\npart1:",p1(),1751)


start

part1: 1751 1751
CPU times: user 14.7 ms, sys: 0 ns, total: 14.7 ms
Wall time: 14.6 ms


In [2]:
%%time
from more_itertools import set_partitions
import copy

def p2():
    M,F=ld()
    keys=M.keys()
    cbs=set_partitions(keys,2)
    
    mmm=0
    for a,b in cbs:
        r=0
        r+=maxrel(M,F,26,a)
        r+=maxrel(M,F,26,b)
        if r>mmm:
            mmm=r
    return mmm
        
        


print("\npart2:",p2(),2207)




part2: 2207 2207
CPU times: user 28.7 s, sys: 0 ns, total: 28.7 s
Wall time: 28.7 s
